In [8]:
import pandas as pd 
import joblib
import random
import extra_funcs_TO
import time
import math
#
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
#
from matplotlib import pyplot as plt
from pandas.tools.plotting import table
import subprocess
%matplotlib inline
import re
from functools import reduce
from tqdm import tqdm
import pandas as pd
import os
import collections
import scipy.stats
import numpy as np
import scipy
import csv
import math
from scipy import interp
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, auc
from sklearn import svm
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_recall_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
import subprocess
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import re
import seaborn as sns
from functools import reduce
from sklearn.model_selection import LeaveOneOut
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.utils.multiclass import unique_labels

In [9]:
### READ DATA
rawData = pd.read_csv("mydata.csv", low_memory=False)
Hchain = rawData["sequence_alignment_aa_heavy"]
Lchain = rawData["sequence_alignment_aa_light"]

In [10]:
Lchain.head()

0    DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKP...
1    QSVLTQPPSVSAAPGQKVTISCSGSSSNIGNNYVSWYQQLPGTAPK...
2    QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNYVYWYQQLPGTAPK...
3    EIVMTQSPATLSVSPGERATLSCRASQSVSSNLAWYQQKPGQAPRL...
4    EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...
Name: sequence_alignment_aa_light, dtype: object

In [11]:
### TRUNCATE FOR DEV PURPOSES
Hchain = Hchain[:1000]
Lchain = Lchain[:1000]

In [12]:
### SHUFFLE LCHAIN
LchainShuf = Lchain.sample(frac=1)

In [13]:
# ALIGN DATA
aligned = extra_funcs_TO.align_abs(Hchain, Lchain)
alignedShuf = extra_funcs_TO.align_abs(Hchain, LchainShuf)
alignedShuf = alignedShuf[~alignedShuf.isin(["no seq"])]
alignedShuf.dropna(inplace=True)

In [14]:
# ENDCODE
encoded = extra_funcs_TO.one_hot_encode(aligned.Hchain_align, aligned.Lchain_align)
encodedShuf = extra_funcs_TO.one_hot_encode(alignedShuf.Hchain_align, alignedShuf.Lchain_align)

### ADD LABEL
encoded["pairing"] = 1
encodedShuf["pairing"] = 0

### MERGE FRAMES
data = pd.concat([encoded, encodedShuf])

### X, y
X = data.drop("pairing", axis=1)
y = data["pairing"]
# X_train, X_test, y_train, y_test = train_test_split(data.drop("pairing", axis=1), data["pairing"], test_size=0.30,stratify=y)


In [15]:
############## MODEL SELECTION ##############
inner_cv = 10
gridSearchScorer = "accuracy"  #"recall"
# CV for cross_validate scoring
outer_cv = 10

# Scoring metric for cross_validate
scoring = {"acc":"accuracy",
           "b_acc":"balanced_accuracy",
           "recall":"recall",
           "precision":"precision",
           "f1_micro":"f1_micro",
           "f1_macro":"f1_macro",
           "f1_weighted":"f1_weighted",
           "roc_auc":"roc_auc",
           "matthews_corrcoef":make_scorer(matthews_corrcoef)}

# Master Switch
do_logisticRegression = True
do_svc = False
do_knn = False
do_randomForest = False
do_MLP = False


In [ ]:
from sklearn.svm import SVC
t0 = time.time()
print("\nSVC \t initiate ...\n")
# Construct Pipe
svc_clf = Pipeline([("classifier", SVC(random_state=None))])
# Define hyperparameter space
svc_param_grid = [{"classifier__C": [0.01, 0.1, 1],
                   "classifier__kernel":["poly", "rbf"],
                   "classifier__degree":range(2,5)}]
# Define hyperparameter space
svc_gs = GridSearchCV(svc_clf, svc_param_grid,
                     cv = inner_cv,
                     n_jobs=-1,
                     scoring=gridSearchScorer,
                     verbose=0)
# svc_gs.fit(X_train, y_train)
# svc_gs = svc_gs.cv_results_
# print("Best estimator: {}".format(svc_gs.best_estimator_["classifier"]))
# Get scoring metrics
print("SVC \t scoring ...\n")
svc_score = cross_validate(svc_gs, X, y, cv=outer_cv, n_jobs=-1,
                       scoring=scoring,
                       verbose=0) 
svc_score_df = pd.DataFrame(svc_score)
# Save as .csv
print("SVC \t saving results ...\n")
svc_score_df.to_csv("results_SVC_2500.csv")
t1 = time.time() - t0
print("SVC \t finished!!! (Time needed: {}) \n".format(t1))



SVC 	 initiate ...

SVC 	 scoring ...



In [ ]:
import joblib
joblib.dump(svc_clf, "finalized_model_1000.sav")